In [1]:
!nvidia-smi

Thu Jun  1 05:19:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090       WDDM | 00000000:01:00.0  On |                  N/A |
|  0%   30C    P8               41W / 350W|   1791MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [4]:
def num_gpus():  #@save
    """Get the number of available GPUs."""
    return torch.cuda.device_count()

num_gpus()

2

In [5]:
torch.cuda.device_count()

2

In [6]:
torch.cuda.is_available()

True

In [7]:
a=torch.cuda.FloatTensor()

In [8]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [10]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

In [11]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [12]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.8046, 0.6867, 0.2542],
        [0.0946, 0.8602, 0.8861]], device='cuda:1')

In [13]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


In [14]:
Y + Z

tensor([[1.8046, 1.6867, 1.2542],
        [1.0946, 1.8602, 1.8861]], device='cuda:1')

In [15]:
Z.cuda(1) is Z

True

In [16]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())


C:\Users\david\miniconda3\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [17]:
net(X)

tensor([[0.8585],
        [0.8585]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [18]:
net[0].weight.data.device

device(type='cuda', index=0)